In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.models import mobilenet_v2
import pickle
from sklearn.model_selection import train_test_split
from helper_functions import train_or_load_and_eval_atck_model, create_eval_post_loader, create_shadow_post_train_loader, evaluate_attack_model, DatasetClassN

Parameters

In [2]:
# Model Parameter
model = "mobilenetv2"
model_short = "mobilenet"
MODEL_MODULE = mobilenet_v2
# Dataset parameter
dataset = "cifar10"
dataset_short = "cifar"
DATASET_ENUM = DatasetClassN.Cifar

## Attack model training parameters
LEARNING_R = 0.001
EPOCHS = 3
MULTI_WORKERS_N = 1

## Datasets
SHADOW_DATA_PATH = f"pickle/{dataset}/{model}/shadow.p"
EVALUATE_DATA_PATH = f"pickle/{dataset}/{model}/eval.p"
# Save Dset create for attack model training
ATT_TRAIN_DATA_PATH = f"pickle/{dataset}/{model}/attack_train.p"
## Models
SHADOW_MODEL_PATH = f"shadow_models/{model}_shadow_{dataset_short}_overtrained.pth"
TARGET_MODEL_PATH = f"models/{model}_{dataset}.pth"
ATTACK_MODEL_PATH = f"attack_models/attack_{model_short}_{dataset_short}.pth"

DEVICE=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Shadow Model

Shadow Model

In [3]:
#load the shadow model trained in the other python script
shadow_model = MODEL_MODULE(weights=None,num_classes=DATASET_ENUM.value).to(DEVICE) #resnet_target is the shadow model
shadow_model.load_state_dict(torch.load(SHADOW_MODEL_PATH, map_location=DEVICE))
pass

Shadow dataset

In [4]:
with open(SHADOW_DATA_PATH, "rb") as f:
    dataset = pickle.load(f)
    
shadow_memb_data, shadow_non_memb_data = train_test_split(dataset, test_size=(1-0.5),shuffle=False)
  
shadow_membloader = DataLoader(shadow_memb_data, batch_size=1, shuffle=False, num_workers=1)
shadow_non_membloader =  DataLoader(shadow_non_memb_data, batch_size=1, shuffle=True, num_workers=1)


### Attack Training Dataset

Create Attack model training dataset

In [5]:
attack_train_loader = create_shadow_post_train_loader(shadow_non_membloader, shadow_membloader, shadow_model, batch_size=64, \
    multi_n= MULTI_WORKERS_N, device=DEVICE, save_path=ATT_TRAIN_DATA_PATH)

NON Members
------SAMPLE WINDOW---------------------------------------------------------
Number Samples: 15000
Batchsize: 1
Inputs:tensor([[[[0.4667, 0.5098, 0.6784,  ..., 0.4118, 0.5216, 0.5412], [0.4863, 0.5451, 0.6196,  ..., 0.3922, 0.5333, 0.5647], [0.5176, 0.4863, 0.4745,  ..., 0.4196, 0.5490, 0.6157], ..., [0.5529, 0.6157, 0.6392,  ..., 0.6863, 0.6667, 0.6157], [0.6510, 0.6000, 0.5725,  ..., 0.6784, 0.7412, 0.5333], [0.7059, 0.6196, 0.5569,  ..., 0.5412, 0.7137, 0.5294]],[[0.3725, 0.4078, 0.5765,  ..., 0.3490, 0.4431, 0.4627], [0.4157, 0.4471, 0.4980,  ..., 0.3098, 0.4275, 0.4588], [0.4588, 0.3961, 0.3569,  ..., 0.3255, 0.4392, 0.5020], ..., [0.4824, 0.5294, 0.5373,  ..., 0.5922, 0.5686, 0.4588], [0.5412, 0.4824, 0.4510,  ..., 0.6000, 0.6392, 0.3961], [0.5804, 0.5020, 0.4431,  ..., 0.4627, 0.6078, 0.4118]],[[0.4275, 0.4196, 0.5490,  ..., 0.3490, 0.4431, 0.4588], [0.4588, 0.4667, 0.4941,  ..., 0.3176, 0.4431, 0.4745], [0.4980, 0.4275, 0.3765,  ..., 0.3451, 0.4706, 0.5333], ..., [0

In [6]:
attack_sample_iter = iter(attack_train_loader)
print (str(next(attack_sample_iter)).replace('\n',"").replace("   ",""))

[tensor([[[ 0.3324,  0.2683,  0.2625]],  [[ 0.1879,  0.1779,  0.1584]],  [[ 0.4012,  0.3404,  0.2902]],  [[ 0.8235,  0.6690,  0.4599]],  [[ 0.4708,  0.4280,  0.3601]],  [[ 0.1084,  0.1064,  0.0687]],  [[ 0.4322,  0.3813,  0.2232]],  [[ 0.8849,  0.5162,  0.4315]],  [[ 0.7891,  0.5271,  0.3867]],  [[ 0.6269,  0.2678,  0.2178]],  [[ 0.1874,  0.1324,  0.0918]],  [[ 0.4585,  0.3624,  0.2958]],  [[ 0.2964,  0.0958,  0.0332]],  [[ 0.1557,  0.1431,  0.0837]],  [[ 0.8102,  0.3505,  0.0933]],  [[ 0.1455,  0.1326,  0.0930]],  [[ 0.4463,  0.2899,  0.1021]],  [[ 0.8979,  0.7924,  0.5810]],  [[ 0.5377,  0.5223,  0.4898]],  [[ 0.3453,  0.2833,  0.0794]],  [[ 0.3979,  0.2475,  0.1020]],  [[ 0.5987,  0.4747,  0.3113]],  [[ 0.4133,  0.2735,  0.1676]],  [[ 0.5140,  0.4178, -0.0517]],  [[ 1.5245,  0.5602,  0.2532]],  [[ 0.6157,  0.5370,  0.3280]],  [[ 0.2527,  0.1128,  0.0946]],  [[ 0.6565,  0.2377,  0.1826]],  [[ 0.5292,  0.4166,  0.3680]],  [[ 0.2209,  0.0861,  0.0502]],  [[ 0.2491,  0.2023,  0.1803]], 

### Attack Model

Simple Model

In [7]:
class SmallAttackNN(nn.Module):
    def __init__(self):
        super(SmallAttackNN, self).__init__()
        self.fc1 = nn.Linear(3, 32)
        self.fc2 = nn.Linear(32, 1)
       

    def forward(self, x):
        
        x = torch.sigmoid(self.fc2(self.fc1(x)))
        return x

Middle Size Model

In [8]:
class MiddleAttackNN(nn.Module):
    def __init__(self):
        super(MiddleAttackNN, self).__init__()
        self.fc1 = nn.Linear(3, 32)
        self.bn1 = nn.BatchNorm1d(32)
        self.fc2 = nn.Linear(32, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)
        self.fc4 = nn.Linear(32, 16)
        self.bn4 = nn.BatchNorm1d(16)
        self.fc5 = nn.Linear(16, 8)
        self.bn5 = nn.BatchNorm1d(8)
        self.fc6 = nn.Linear(8, 1)
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()
       

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = torch.relu(self.bn3(self.fc3(x)))
        x = torch.relu(self.bn4(self.fc4(x)))
        x = self.dropout(torch.relu(self.bn5(self.fc5(x))))
        x = self.sigmoid(self.fc6(x))
        return x

Load Target Model

In [9]:
# Load target model
target_model = MODEL_MODULE(weights=None,num_classes=DATASET_ENUM.value)
target_model.load_state_dict (torch.load (TARGET_MODEL_PATH, map_location=DEVICE)["net"])
target_model.eval()
target_model.to(DEVICE)
pass

### Evaluation Dataset

Load **Evaluation Dataset** & get Posteriors/ Member Labels 

In [10]:
with open(EVALUATE_DATA_PATH, "rb") as eval_f:
    eval_dataset = pickle.load(eval_f)
    # Create Posteriors with target model; MULTI_WORKERS_N defines workers num of returned DL
    attack_eval_post_loader = create_eval_post_loader (target_model, eval_dataset, MULTI_WORKERS_N, DEVICE)

------SAMPLE WINDOW---------------------------------------------------------
Number Samples: 200
Batchsize: 1
Inputs:tensor([[[[0.8941, 0.9216, 0.9294,  ..., 0.1765, 0.4000, 0.7059], [0.9451, 0.9569, 0.9765,  ..., 0.1882, 0.6667, 0.6157], [0.8941, 0.9569, 0.9843,  ..., 0.1373, 0.3961, 0.3961], ..., [0.2784, 0.2510, 0.2392,  ..., 0.3412, 0.3137, 0.2941], [0.3961, 0.3333, 0.2392,  ..., 0.3137, 0.2980, 0.2745], [0.4275, 0.3647, 0.2353,  ..., 0.3137, 0.3020, 0.3020]],[[0.9882, 0.9843, 0.9882,  ..., 0.2667, 0.5176, 0.8078], [1.0000, 0.9961, 1.0000,  ..., 0.2667, 0.7647, 0.7059], [0.9216, 0.9804, 0.9922,  ..., 0.1961, 0.5020, 0.5216], ..., [0.3059, 0.3137, 0.3216,  ..., 0.4078, 0.3725, 0.3176], [0.4118, 0.3882, 0.3098,  ..., 0.4039, 0.3686, 0.3176], [0.4471, 0.3882, 0.2745,  ..., 0.3608, 0.3451, 0.3451]],[[0.9882, 0.9843, 0.9882,  ..., 0.3137, 0.6392, 0.8196], [1.0000, 0.9961, 1.0000,  ..., 0.2863, 0.7843, 0.6549], [0.9294, 0.9843, 0.9922,  ..., 0.2353, 0.5137, 0.4863], ..., [0.2353, 0.2510,

Attack Evaluation DL Sample

In [11]:
eval_sample_iter = iter(attack_eval_post_loader)
print (str(next(eval_sample_iter)).replace('\n',"").replace("   ",""))

[tensor([[[ 1.0237e+01,  5.1730e-01, -3.0826e-01]],  [[ 9.4312e+00,  5.0981e-01, -2.7521e-01]],  [[ 1.1062e+01,  6.9038e-01, -4.8954e-01]],  [[ 8.8281e+00,  7.7090e-01,  1.4212e-01]],  [[ 9.5954e+00,  8.7125e-01,  2.9888e-01]],  [[ 8.4472e+00,  1.1986e-01,  1.0038e-01]],  [[ 1.0364e+01, -6.1306e-02, -1.0229e-01]],  [[ 8.5466e+00,  4.6691e-01, -1.5146e-02]],  [[ 1.0357e+01,  6.2545e-01, -3.0423e-01]],  [[ 9.2281e+00, -1.7976e-01, -4.3510e-01]],  [[ 9.9767e+00,  6.5998e-02, -1.9517e-01]],  [[ 7.6929e+00,  4.7245e-01,  3.0626e-01]],  [[ 9.2761e+00,  5.8081e-01, -5.9289e-02]],  [[ 8.9620e+00, -1.0565e-01, -3.5072e-01]],  [[ 1.0706e+01,  2.9947e-01, -7.3507e-02]],  [[ 8.6948e+00,  1.1925e+00,  4.0340e-01]],  [[ 1.0445e+01,  3.0757e-01, -3.0828e-01]],  [[ 7.6198e+00,  5.4609e-01,  2.5895e-02]],  [[ 7.7251e+00,  6.8454e-01,  2.0549e-02]],  [[ 9.8122e+00,  9.1789e-01, -3.0138e-01]],  [[ 7.3903e+00,  3.4802e-01,  1.9170e-01]],  [[ 8.6537e+00,  3.9655e-01,  3.0257e-02]],  [[ 8.6094e+00,  2.4598e

### Train Attack model

In [12]:
attack_model = MiddleAttackNN()
attack_model.to(DEVICE)
# Further training Parameters 
criterion = nn.BCELoss()
optimizer = optim.Adam(attack_model.parameters(), lr=LEARNING_R)
# Do everything in one function
# Train/ Load Attack model; For every epoch show epoch loss and evaluate
train_or_load_and_eval_atck_model(attack_model, attack_train_loader, attack_eval_post_loader, optimizer, criterion, EPOCHS, ATTACK_MODEL_PATH, DEVICE)


Epoch: 1
------SAMPLE WINDOW---------------------------------------------------------
Number Samples: 30000
Batchsize: 64
Inputs:tensor([[[ 0.7274,  0.4342,  0.4001]],  [[ 0.3136,  0.1517,  0.0857]],  [[ 0.8958,  0.2023,  0.1934]],  [[ 0.2576,  0.2556,  0.1700]],  [[ 0.3290,  0.3031,  0.2512]],  [[ 0.2480,  0.1710,  0.0838]],  [[ 1.1055,  0.9081,  0.7799]],  [[ 1.0919,  0.8687,  0.5026]],  [[ 0.6131,  0.5213,  0.4387]],  [[ 2.8887,  1.1767,  0.7088]],  [[ 0.4040,  0.2605,  0.2128]],  [[ 0.2461,  0.1780,  0.1391]],  [[ 1.5014,  0.8427,  0.5380]],  [[ 0.4186,  0.2661,  0.1282]],  [[ 0.4907,  0.4440,  0.3915]],  [[ 0.6676,  0.6279,  0.4062]],  [[ 0.5092,  0.3870,  0.2943]],  [[ 1.4139,  0.9389,  0.5323]],  [[ 0.5645,  0.2960,  0.2231]],  [[ 0.4151,  0.3467,  0.1358]],  [[ 0.2971,  0.2568,  0.1571]],  [[ 0.5526,  0.3743,  0.1208]],  [[ 0.3083,  0.2585,  0.2136]],  [[ 0.2465,  0.2235,  0.1322]],  [[ 0.6318,  0.5664,  0.3446]],  [[ 0.7091,  0.2397,  0.2029]],  [[ 1.5267,  0.7640,  0.1854]], 

### Evaluation

In [13]:
evaluate_attack_model(attack_model, attack_eval_post_loader, DEVICE)

Accuracy: 33.60
